In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data_path_resort = './data/H1.xlsx'
data_path_city = './data/H2.xlsx'

In [6]:
df_h1 = pd.read_excel(data_path_resort)
df_h1

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0,...,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0,...,No Deposit,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,2017,August,35,31,2,8,2,1,...,No Deposit,143,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10
40056,0,169,2017,August,35,30,2,9,2,0,...,No Deposit,250,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10
40057,0,204,2017,August,35,29,4,10,2,0,...,No Deposit,250,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12
40058,0,211,2017,August,35,31,4,10,2,0,...,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14


In [11]:
## Check for constant features
df_h1.nunique().sort_values(ascending=True)

IsCanceled                        2
IsRepeatedGuest                   2
ArrivalDateYear                   3
DepositType                       3
Babies                            3
ReservationStatus                 3
CustomerType                      4
DistributionChannel               4
RequiredCarParkingSpaces          5
Children                          5
Meal                              5
TotalOfSpecialRequests            6
MarketSegment                     6
ReservedRoomType                 10
AssignedRoomType                 11
PreviousCancellations            11
ArrivalDateMonth                 12
Adults                           14
BookingChanges                   15
StaysInWeekendNights             16
PreviousBookingsNotCanceled      31
StaysInWeekNights                31
ArrivalDateDayOfMonth            31
DaysInWaitingList                44
ArrivalDateWeekNumber            53
Country                         125
Agent                           186
Company                     